In [24]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0
buy = 0
sell = 0

In [25]:
df = pd.read_csv('BTC_15min_10JAN.csv')

In [26]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['BTC']

nstd = 2
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

def diverg(df,i,symbol):
    p = i - 2
    while p > i - 100:
        if df[f'{symbol}-USD_Close'].iloc[p] < df[f'{symbol}-USD_Open'].iloc[p]:
            if df[f'{symbol}-USD_Close'].iloc[i] < df[f'{symbol}-USD_Close'].iloc[p] and df[f'{symbol}_rsi'].iloc[i] > df[f'{symbol}_rsi'].iloc[p]:
                return True
            else: 
                p = p - 1
        else: 
            p = p - 1


for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [27]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.divs = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
    
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'
        
# VIP level 0, paying fees with BNB = 0.075%

        

In [28]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1) #0.99925

In [29]:
last_buy = 0

for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                if diverg(df, i, symbol): 
                 buy = 1
            if  buy == 1 and df[f'{symbol}-USD_Close'].iloc[i] > df[f'{symbol}-USD_Open'].iloc[i]:
                env.buy(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                last_buy = df[f'{symbol}_lower_band'].iloc[i]
                buy = 0
                break
            
    if env.balance_unit != 'USDT':
        if last_buy*0.98 > df[f'{env.balance_unit}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
            sell = 0
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]: #sell signal
            sell = 1
        if sell == 1 and df[f'{symbol}-USD_Close'].iloc[i] < df[f'{symbol}-USD_Open'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
            sell = 0
            
if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [30]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print('starting balance: 100 USDT')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 603
num sells: 603
starting balance: 100 USDT
ending balance: 57.456223004845256 USDT


In [23]:
balance = 100
profit_total = 0
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(f'buys = {buys}')
    print(f'sells = {sells}')
    print('P/L = 'f'{profit}')
    balance += balance*(profit/100)
    print(balance)
    print('\n')

buys = ['BTC', '2022-01-11 23:59:59.999', 41701.52]
sells = ['BTC', '2022-01-12 01:29:59.999', 41802.05]
P/L = 0.2410703494740929
100.24107034947409


buys = ['BTC', '2022-01-12 01:59:59.999', 41733.93]
sells = ['BTC', '2022-01-12 02:59:59.999', 42705.24]
P/L = 2.327386852855695
102.57406784194957


buys = ['BTC', '2022-01-13 22:14:59.999', 43776.93]
sells = ['BTC', '2022-01-14 00:44:59.999', 44177.66]
P/L = 0.915390823431439
103.51302144619511


buys = ['BTC', '2022-01-14 09:14:59.999', 42661.63]
sells = ['BTC', '2022-01-14 14:59:59.999', 42680.28]
P/L = 0.04371609804876275
103.55827330014377


buys = ['BTC', '2022-01-14 18:29:59.999', 42477.83]
sells = ['BTC', '2022-01-15 01:44:59.999', 42942.4]
P/L = 1.0936763954279227
104.69086569074018


buys = ['BTC', '2022-01-15 10:29:59.999', 43055.76]
sells = ['BTC', '2022-01-15 20:29:59.999', 43196.52]
P/L = 0.3269248992469276
105.03312619792037


buys = ['BTC', '2022-01-15 22:59:59.999', 42715.56]
sells = ['BTC', '2022-01-16 00:59:59.999', 4